In [2]:
import os
import json
from anthropic import Anthropic
from anthropic.types.message_create_params import MessageCreateParamsNonStreaming
from anthropic.types.messages.batch_create_params import Request
from pathlib import Path

In [3]:
client = Anthropic(
    api_key=os.environ.get("ANTHROPIC_API_KEY"),  # This is the default and can be omitted
)

message = client.messages.create(
    max_tokens=1024,
    messages=[
        {
            "role": "user",
            "content": "Hello, Claude",
        }
    ],
    model="claude-3-5-sonnet-latest",
)
print(message.content)

OverloadedError: Error code: 529 - {'type': 'error', 'error': {'type': 'overloaded_error', 'message': 'Overloaded'}}

In [3]:
result = client.beta.messages.batches.create(
    requests=[
        {
            "custom_id": "my-first-request",
            "params": {
                "model": "claude-3-5-sonnet-latest",
                "max_tokens": 1024,
                "messages": [{"role": "user", "content": "Hello, world"}],
            },
        },
        {
            "custom_id": "my-second-request",
            "params": {
                "model": "claude-3-5-sonnet-latest",
                "max_tokens": 1024,
                "messages": [{"role": "user", "content": "Hi again, friend"}],
            },
        },
    ]
)


In [4]:
for message_batch in client.messages.batches.list(limit=20):
    print(message_batch)

MessageBatch(id='msgbatch_01GPZ5YrTCg73xJoLoNCRcmg', archived_at=None, cancel_initiated_at=None, created_at=datetime.datetime(2025, 7, 3, 1, 49, 0, 674555, tzinfo=datetime.timezone.utc), ended_at=datetime.datetime(2025, 7, 3, 1, 50, 22, 917170, tzinfo=TzInfo(UTC)), expires_at=datetime.datetime(2025, 7, 4, 1, 49, 0, 674555, tzinfo=datetime.timezone.utc), processing_status='ended', request_counts=MessageBatchRequestCounts(canceled=0, errored=0, expired=0, processing=0, succeeded=6), results_url='https://api.anthropic.com/v1/messages/batches/msgbatch_01GPZ5YrTCg73xJoLoNCRcmg/results', type='message_batch')
MessageBatch(id='msgbatch_017seTsRug5vNHJcTChmpNcZ', archived_at=None, cancel_initiated_at=None, created_at=datetime.datetime(2025, 7, 3, 1, 44, 42, 857241, tzinfo=datetime.timezone.utc), ended_at=datetime.datetime(2025, 7, 3, 1, 45, 43, 247876, tzinfo=TzInfo(UTC)), expires_at=datetime.datetime(2025, 7, 4, 1, 44, 42, 857241, tzinfo=datetime.timezone.utc), processing_status='ended', requ

In [39]:
path = Path("/Users/williamthompson/Code/kellogg/llm-batch/notebooks/requests/claude-sonnet-4-20250514")
request_files = list(path.glob("*.json"))

In [59]:
batch_requests = Path("/Users/williamthompson/Code/kellogg/llm-batch/notebooks/batch-requests.jsonl") 
request_datas = []
with open(batch_requests, "r") as f:
    for line in f:
        request_datas.append(json.loads(line))

In [5]:
requests=[]
for idx, request_data in enumerate(request_datas):
    body = request_data['body']
    params=MessageCreateParamsNonStreaming(
        model=body['model'],
        max_tokens=body['max_tokens'],
        messages=body['messages'],
    )
    requests.append(
        Request(
            custom_id=f"id-{idx}", 
            params=params
        )
    )

NameError: name 'request_datas' is not defined

In [64]:
requests[0]

{'custom_id': 'id-0',
 'params': {'model': 'claude-sonnet-4-20250514',
  'max_tokens': 15,
  'messages': [{'role': 'user',
    'content': [{'text': 'Below is a summary of a study from an academic paper:\n \nThe study aimed to test how two different drugs impact whether a patient recovers from a certain disease. Subjects were randomly drawn from a fixed population and then randomly assigned to Drug A or Drug B. Fifty-two percent (52%) of subjects who took Drug A recovered from the disease while forty-four percent (44%) of subjects who took Drug B recovered from the disease.\n \nA test of the null hypothesis that there is no difference between Drug A and Drug B in terms of probability of recovery from the disease yields a *p*-value of 0.0.51.\n \nAssuming no prior studies have been conducted with these drugs, which of the following statements is most accurate\n\nOption A. A person drawn randomly from the same population as the subjects in the study is **more likely** to recover from the 

In [65]:
message_batch = client.messages.batches.create(requests=requests)

In [6]:
for batch in client.messages.batches.list(
    limit=20
):
    print(batch)

MessageBatch(id='msgbatch_01GPZ5YrTCg73xJoLoNCRcmg', archived_at=None, cancel_initiated_at=None, created_at=datetime.datetime(2025, 7, 3, 1, 49, 0, 674555, tzinfo=datetime.timezone.utc), ended_at=datetime.datetime(2025, 7, 3, 1, 50, 22, 917170, tzinfo=TzInfo(UTC)), expires_at=datetime.datetime(2025, 7, 4, 1, 49, 0, 674555, tzinfo=datetime.timezone.utc), processing_status='ended', request_counts=MessageBatchRequestCounts(canceled=0, errored=0, expired=0, processing=0, succeeded=6), results_url='https://api.anthropic.com/v1/messages/batches/msgbatch_01GPZ5YrTCg73xJoLoNCRcmg/results', type='message_batch')
MessageBatch(id='msgbatch_017seTsRug5vNHJcTChmpNcZ', archived_at=None, cancel_initiated_at=None, created_at=datetime.datetime(2025, 7, 3, 1, 44, 42, 857241, tzinfo=datetime.timezone.utc), ended_at=datetime.datetime(2025, 7, 3, 1, 45, 43, 247876, tzinfo=TzInfo(UTC)), expires_at=datetime.datetime(2025, 7, 4, 1, 44, 42, 857241, tzinfo=datetime.timezone.utc), processing_status='ended', requ

In [7]:
results = client.messages.batches.results(
    message_batch_id="msgbatch_017seTsRug5vNHJcTChmpNcZ",
)
for result in results:
    print(result.to_json())

{
  "custom_id": "id-0",
  "result": {
    "message": {
      "id": "msg_01V3maWLoZK5oEU2RPi7GYfZ",
      "content": [
        {
          "text": "Looking at this study:\n\n- Drug A: 52% recovery rate",
          "type": "text"
        }
      ],
      "model": "claude-sonnet-4-20250514",
      "role": "assistant",
      "stop_reason": "max_tokens",
      "stop_sequence": null,
      "type": "message",
      "usage": {
        "cache_creation_input_tokens": 0,
        "cache_read_input_tokens": 0,
        "input_tokens": 411,
        "output_tokens": 15,
        "service_tier": "batch"
      }
    },
    "type": "succeeded"
  }
}
{
  "custom_id": "id-1",
  "result": {
    "message": {
      "id": "msg_01XhFP2eDHaEA1USUsnu1z6W",
      "content": [
        {
          "text": "Option D.\n\nThe p-value of 0.49 indicates",
          "type": "text"
        }
      ],
      "model": "claude-sonnet-4-20250514",
      "role": "assistant",
      "stop_reason": "max_tokens",
      "stop_sequenc